# FUNCIONES

In [1]:
import os
import numpy as np
import pprint
import copy
from math import sqrt
from scipy.linalg import solve_triangular

### Creación matrices

In [2]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

### Factorización QR

In [3]:
def house(x):
    """
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
    """
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

In [4]:
def factorizacion_QR(A):
    """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: A_r_v  Matriz (mxn) con la información escencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
    """
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [5]:
def QT_C(A_r_v,C):
    """
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: M       Matriz con numero al azar
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [6]:
def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

### Funciones para solución de Sistemas de Ecuaciones Lineales

In [7]:
def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    A_r_v=factorizacion_QR(A)
    m=A_r_v.shape[0]
    #Q=np.transpose(QT_C(A_r_v,np.eye(m)))
    #R=np.transpose(Q)@A
    n=A_r_v.shape[0]
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(m):
        Qj=Q_j(A_r_v,j)
        Q=Q@Qj
        R=Q_j(A_r_v,j)@R
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x

#### Eliminación por bloques

In [8]:
def bloques(A, b=False, n1=False, n2=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')

    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [9]:
def eliminacion_bloques(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        ys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

# Prueba Unitaria

## Eliminación por bloques con QR considerando sistemas con soluciones infinitas

### Ejemplo 1 - Matriz 2 x 2 (Ecuaciones linealmente dependientes)

El siguiente sistema representa un sistema de 2 ecuaciones con 2 incógnitas. La segunda ecuación es una combinación lineal de la primera, por lo que al multiplicar la segunda ecuación por -2 se obtiene la primera.

In [10]:
# Generación de lado izquierdo A y lado derecho b de un sistema de ecuaciones de 2x2

A = np.array([[-6, 4], [3, -2]], dtype='d')
b = np.array([[2], [-1]], dtype='d')
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b)

A:
array([[-6.,  4.],
       [ 3., -2.]])
b:
array([[ 2.],
       [-1.]])


**Numpy**

Puesto que el sistema tiene infinitas soluciones, se utilizará de la función *linalg.solve(A,b)*, perteneciente a la librería numpy para conocer el error que marca al ingresar tal sistema. 

In [11]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

Puntualmente, el error indicado por la función *linalg.solve* indica que la matriz A es singular, lo que implica que no sea posible obtener la matriz inversa y por tanto no se puede resolver el sistema.

**Implementación Programadores - Eliminación por bloques con QR**

La función *eliminacion_bloques*, implementada por los programadores permite resolver un sistema de ecuaciones lineales. Se revisará el error que despliega al intentar resovler el sistema de ecuaciones propuesto.

In [12]:
eliminacion_bloques(A,b)

NameError: name 'sys' is not defined

Lo anterior muestra que la función *eliminacion_bloques* validó si el determinante de la matriz A era igual cero, lo cual es equivalente a evaluar si la matriz es *no singular*. Como lo condición no fue cumplida, es decir, la matriz es singular, entonces la función ya no es completada. 

El error que marca esta función es consistente con el desplegado por la función *linalg.solve*.

### Ejemplo 2 - Matriz $10^{2}$ x $10^{2}$ (Matriz de unos)

Se genera un sistema de ecuaciones lineales de $10^{2} x 10^{2}$, cuyo lado izquierdo es una matriz de 1's y el lado derecho es un vector que contiene al número 100 en cada una de sus entradas. Este sistema representa la misma ecuación, repetida 100 veces.

In [14]:
# Generación lado izquierdo A (matriz de 1's) y lado derecho b (vector con entradas igual a 100)

m = 10**2
n = 10**2
A = crea_matriz(m, n, 1, 1,True)
b = crea_matriz(m, 1, 100, 100,True)
print("A:")
pprint.pprint(A)
print("b:")
pprint.pprint(b[0:10,:])

A:
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])
b:
array([[100.],
       [100.],
       [100.],
       [100.],
       [100.],
       [100.],
       [100.],
       [100.],
       [100.],
       [100.]])


**Numpy**

Nuevamente se hará uso de la función *linalg.solve(A,b)* de la librería numpy para conocer qué error despliega al intentar resolver el sistema propuesto.

In [15]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

De igual forma que en el Ejemplo 1, el error indica que la matriz A es singular.

**Implementación Programadores - Eliminación por bloques con QR**

Se implementa la función *eliminacion_bloques* creada por los programadores para verificar que se obtenga el mismo error que marca al función *linalg.solve(A,b)*.

In [16]:
eliminacion_bloques(A,b)

NameError: name 'sys' is not defined

Efectivamente, la función *eliminacion_bloques* indica que la matriz A es singular.

### Ejemplo 3 - Matriz $10^{2}$ x $10^{2}$ (Matriz de unos, sistema homogéneo)

De forma análoga al Ejemplo 2, se genera un sistema de ecuaciones lineales de $10^{2} x 10^{2}$, cuyo lado izquierdo es una matriz de 1's; sin embargo el lado derecho es un vector que contiene al número 0 en cada una de sus entradas, es decir, se trata de un sistema homogéneo. De igual forma, este sistema representa la misma ecuación, repetida 100 veces.

In [18]:
# Generación lado izquierdo A (matriz de 1's) y lado derecho b (vector con entradas igual a 0)

m = 10**2
n = 10**2
A = crea_matriz(m, n, 1, 1,True)
b = crea_matriz(m, 1, 0, 0,True)
print("A:")
pprint.pprint(A)
print("b:")
# se muestran los primeros 10 elementos del lado derecho b
pprint.pprint(b[0:10,:]) 

A:
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])
b:
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])


**Numpy**

Se intenta resolver el sistema con *linalg.solve(A,b)*:

In [19]:
np.linalg.solve(A,b)

LinAlgError: Singular matrix

*linalg.solve(A,b)* despliega el mismo error que en los Ejemplos 1 y 2.

**Implementación Programadores - Eliminación por bloques con QR**

Finalmente se ejecuta la función *eliminacion_bloques* implementada por los programadores.

In [20]:
eliminacion_bloques(A,b)

NameError: name 'sys' is not defined

Tal como se esperaba, *eliminacion_bloques* indica que A debe ser una matriz no singular.

Los Ejemplos 2 y 3 demuestran que la solución de un sistema de ecuaciones es independiente del lado derecho $b$. En estos ejemplos se mantuvo el mismo lado izquierdo $A$ y se varió el lado derecho $b$, de forma que el Ejemplo 3 es un sistema homogéneo y el Ejemplo 2 no lo es. En ambos casos el error asociado indica que la matriz $A$ es singular. Por lo tanto la solución del sistema de ecuaciones depende en su totalidad del lado izquierdo $A$.